In [1]:
import h5py
import tables
import numpy as np
import pandas as pd
import sys
import os
import bz2
import csv
import math
import arkouda as ak
import timeit
from glob import glob
from decimal import Decimal
from functools import partial
from pathlib import Path

In [45]:
ak.connect(server='localhost',port='5555')

connected to arkouda server tcp://*:5555


In [ ]:
ak.shutdown()

### arkouda

In [29]:
a=[3,5,7]
b=ak.array(a)
b

array([3, 5, 7])

In [55]:
cwd = os.getcwd()
print(glob(cwd+'/dset*.h5'))

['/Users/josephcrandall/Documents/professional/employment/2019/DOD/repos/arkouda-DevOps/notebooks/dset4.h5', '/Users/josephcrandall/Documents/professional/employment/2019/DOD/repos/arkouda-DevOps/notebooks/dset1.h5', '/Users/josephcrandall/Documents/professional/employment/2019/DOD/repos/arkouda-DevOps/notebooks/dset2.h5', '/Users/josephcrandall/Documents/professional/employment/2019/DOD/repos/arkouda-DevOps/notebooks/dset3.h5']


In [57]:
dlist = glob(cwd+'/dset*.h5')
#Causes Segmentation Fault
#dlist = glob(cwd+'/../../data/hdf/*netflow*5.h5')

In [58]:
#ak.read_hdf('SrcPackets',glob('../../data/hdf/*netflow*5.h5'))
#ak.read_hdf('dset',glob('../d*.h5'))
ak.read_hdf('dset',dlist)

array([0, 4, 8, ..., 291, 294, 297])

### Experimental Bellow

In [ ]:
import h5py
h5py.run_tests()

In [ ]:
#bz2.open('../../data/bz2/netflow_day-05.bz2','r')

In [ ]:
df = pd.read_csv('../../data/csv/netflow_day-02.csv')
#df.columns=['Time','Duration','SrcDevice','DstDevice','Protocol','SrcPort','DstPort','SrcPackets','DstPackets','SrcBytes','DstBytes']
#df.to_csv("../../data/csv/netflow_day-05.csv")
print(df.shape)
df.head()

In [ ]:
millnames = ['',' Thousand',' Million', ' Billion', ' Trillion', ' Quadrillion']
def millify(n):
    n=float(n)
    millidx=max(0,min(len(millnames)-1,
                     int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    return '{:.0f}{}'.format(n/10**(3*millidx), millnames[millidx])

In [ ]:
print(millify(1e17))

In [ ]:
#df = pd.DataFrame(data.values,columns=['Time','Duration','SrcDevice','DstDevice','Protocol','SrcPort','DstPort','SrcPackets','DstPackets','SrcBytes','DstBytes'])
#df = pd.DataFrame(data.values, columns = [list(range(11))])
rows, columns = df.shape
#rows = '%.2E' % Decimal(rows)
print(str(millify(rows))+', '+ str(columns))

In [ ]:
df[['SrcPort','DstPort']].head()

In [ ]:
df[['SrcPort','DstPort']].to_hdf('../../data/hdf/SrcPort-netflow_day-05.h5',key='df', mode='w')

In [ ]:
#Series Dataframe
df_Src=pd.read_hdf('../../data/hdf/SrcPort-netflow_day-05.h5', 'df')
print(df_Src.shape)
df_Src.head(5)
#print(str(millify(df_Src.shape[0]))+', '+ str(df_Src.shape[1]))

In [ ]:
hdf = '../../data/hdf'
#print(os.path.join(hdf, 'SrcPort*.h5'))
#print(hdf)
hdfFiles=glob(os.path.join(hdf, 'SrcPort*05.h5'))
#print(type(hdfFiles))
print(hdfFiles)

In [ ]:
#file objext, acts like python dictionary, thus we can check keys -> there is one dataset 'df'
import h5py
f = h5py.File('../../data/hdf/SrcPort-netflow_day-05.h5','r')
list(f.keys())

In [ ]:
filename = '../../data/hdf/SrcPort-netflow_day-05.h5'

with h5py.File(filename, 'r') as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])
    
print(data)

In [ ]:
ak.get_datasets(filename)

In [ ]:
ak.ls_hdf(filename)

In [ ]:
data = ak.read_hdf('df',hdfFiles)

In [ ]:
nrecords=data['srcIP'].size